# Circular Primes
---
### Project Euler #35

The number, 197, is called a circular prime because all rotations of the digits: 197, 971, and 719, are themselves prime.

There are thirteen such primes below 100: 2, 3, 5, 7, 11, 13, 17, 31, 37, 71, 73, 79, and 97.

How many circular primes are there below one million?

In [1]:
import collections

In [2]:
# get the list of primes
def rwh_primes1(n):
    # http://stackoverflow.com/questions/2068372/
    # fastest-way-to-list-all-primes-below-n-in-python/3035188#3035188
    """ Returns  a list of primes < n """
    sieve = [True] * (n//2)
    for i in xrange(3,int(n**0.5)+1,2):
        if sieve[i//2]:
            sieve[i*i//2::i] = [False] * ((n-i*i-1)//(2*i)+1)
    return [2] + [2*i+1 for i in xrange(1,n//2) if sieve[i]]

# fast as fuck!
primes = rwh_primes1(1000000)

In [3]:
# filter out any with even numbers - an even number will eventually 
# cycle to the end of the rotation yielding a non-prime
# this significantly decreases our problem search space
def is_any_even(n):
    digits = [int(i)%2 == 0 for i in list(str(n))]
    if any(digits):
        return True
    else: return False
    
candidate_primes = [2] + [p for p in primes if not is_any_even(p)]

In [4]:
print 'primes: {}'.format(len(primes))
print 'candidate primes: {}'.format(len(candidate_primes))

primes: 78498
candidate primes: 3218


We need a fast way to rotate a number...

In [5]:
# this method is based on the built-in list deque function
# deque is potentially faster but we have to convert the   
# number to a list and join the result
def rotate_deque(n, r):
    d = collections.deque(list(str(n)))
    d.rotate(r)
    return int(''.join(d))

In [6]:
# this method slices a string representation of the number
def rotate_slice(n, r):
    n = str(n)
    return int(n[r:] + n[:r])

In [7]:
%timeit rotate_slice(1234567890, 9)

100000 loops, best of 3: 7.16 µs per loop


In [8]:
%timeit rotate_deque(1234567890, 9)

10000 loops, best of 3: 15.6 µs per loop


The list conversion overhead kills the deque method. Slicing it is!

In [9]:
# were gonna do a ton of membership tests - use a set instead of a list
prime_set = set(primes)

In [10]:
def is_circular_prime(n):
    for i in range(1, len(str(n))):
        if not rotate_slice(n, i) in prime_set:
            return False
    return True

In [11]:
is_circular_prime(123456789)

False

In [12]:
%timeit is_circular_prime(123456789)

100000 loops, best of 3: 9.13 µs per loop


In [13]:
circular_primes = []
for cp in candidate_primes:
    if is_circular_prime(cp):
        circular_primes.append(cp)
        
print len(circular_primes)
print circular_primes

55
[2, 3, 5, 7, 11, 13, 17, 31, 37, 71, 73, 79, 97, 113, 131, 197, 199, 311, 337, 373, 719, 733, 919, 971, 991, 1193, 1931, 3119, 3779, 7793, 7937, 9311, 9377, 11939, 19391, 19937, 37199, 39119, 71993, 91193, 93719, 93911, 99371, 193939, 199933, 319993, 331999, 391939, 393919, 919393, 933199, 939193, 939391, 993319, 999331]


In [14]:
# success!